In [ ]:
import cPickle
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

### Pre-process data

In [ ]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def reshape_img(img):
    img = img.reshape((3,32*32))
    v = np.zeros((32,32,3),dtype=np.uint8)
    for i in range(32):
        for j in range(32):
            v[i][j][0] = img[0][i*32+j]
            v[i][j][1] = img[1][i*32+j]
            v[i][j][2] = img[2][i*32+j]  
    return v


#### Train set

In [ ]:
%%time
DATASET_DIR = "cifar-10-batches-py/"
DATASET_FILES = ["data_batch_1","data_batch_2","data_batch_3","data_batch_4","data_batch_5"]

imgs = []
labels = []

for file in DATASET_FILES:
    data = unpickle(DATASET_DIR + file)
    imgs.extend(data["data"])
    labels.extend(data["labels"])
    
dataset = pd.DataFrame({'img': imgs, 'label': labels})
dataset["img"] = dataset.img.map(reshape_img)
dataset.to_pickle("dataset/train.pkl")

#### Test set

In [ ]:
%%time
DATASET_DIR = "cifar-10-batches-py/"
DATASET_FILES = ["test_batch"]

imgs = []
labels = []

for file in DATASET_FILES:
    data = unpickle(DATASET_DIR + file)
    imgs.extend(data["data"])
    labels.extend(data["labels"])
    
dataset = pd.DataFrame({'img': imgs, 'label': labels})
dataset["img"] = dataset.img.map(reshape_img)
dataset.to_pickle("dataset/test.pkl")

### Load Dataset

In [ ]:
DATASET_DIR = "dataset/"
DATASET_FILE = "train.pkl"

# LABELS = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
LABELS = ['airplane','automobile','bird','cat','frog','horse','ship','truck']

dataset = pd.read_pickle(DATASET_DIR + DATASET_FILE)
dataset = dataset.sample(30000,random_state=7)
dataset = dataset[dataset.label != 4]
dataset = dataset[dataset.label != 5]

#### Train-Test split

In [ ]:
dataset_train, dataset_test = train_test_split(dataset,test_size=.2)

## Method 1: KNN

## Method 2: Linear SVM

In [ ]:
import cv2
from skimage.feature import hog
from skimage import color
from sklearn.svm import LinearSVC,SVC

#### 1). Using HOG descriptors

In [ ]:
%%time
print "Computing descriptors for train set"
X_train = []
for img in dataset_train.img.values:
    X_train.append(hog(color.rgb2gray(img)))
X_train = np.array(X_train)

print "Computing descriptors for test set"
X_test = []
for img in dataset_test.img.values:
    X_test.append(hog(color.rgb2gray(img)))
X_test = np.array(X_test)

y_train = dataset_train.label.values
y_test = dataset_test.label.values

#### 2). Using Kmeans clustered BOW

In [ ]:
DICT_SIZE = 100
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 10)
search_params = dict(checks=20)

bow_trainer = cv2.BOWKMeansTrainer(DICT_SIZE)
sift = cv2.xfeatures2d.SIFT_create()
# matcher = cv2.FlannBasedMatcher(index_params,search_params)
matcher = cv2.BFMatcher(cv2.NORM_L2)
transformer = cv2.BOWImgDescriptorExtractor(sift,matcher)

##### Compute and dump Vocab

In [ ]:
%%time
for i,row in dataset.iterrows():
    kp, des = sift.detectAndCompute(cv2.cvtColor(row.img,cv2.COLOR_BGR2GRAY),None)
#     kp, des = sift.detectAndCompute(row.img,None)
    if des != None:
        bow_trainer.add(des)
    else:
        print "No features found in image: " + str(i)

print "Clustering features.."
vocab = bow_trainer.cluster()
# cPickle.dump(vocab,file("dataset/vocab.pkl","wb"))

##### Load Vocab

In [ ]:
vocab = cPickle.load(file("dataset/vocab.pkl","r"))

##### Get BOW vectors

In [ ]:
%%time

transformer.setVocabulary(vocab)

print "Computing train set BOW vectors.."
X_train = []
for img in dataset_train.img.values:
    v = transformer.compute(img,sift.detect(img,None))
    if v != None:
        X_train.append(np.array(v[0]))
    else:
        X_train.append(np.zeros(DICT_SIZE))
X_train = np.array(X_train)

print "Computing test set BOW vectors.."
X_test = []
for img in dataset_test.img.values:
    v = transformer.compute(img,sift.detect(img,None))
    if v!= None:
        X_test.append(np.array(v[0]))
    else:
        X_test.append(np.zeros(DICT_SIZE))
X_test = np.array(X_test)

y_train = dataset_train.label.values
y_test = dataset_test.label.values

#### Init and Fit Model

In [ ]:
%%time
model = LinearSVC(verbose=1)
# model = SVC(verbose=1)
model.fit(X_train,y_train)

#### Evaluate

In [ ]:
model.score(X_test,y_test)

#### Predict

In [ ]:
preds = model.predict(X_test)

#### Draw confusion matrix

In [ ]:
cm = confusion_matrix(preds,y_test)
df_cm = pd.DataFrame(cm, index = LABELS,
                  columns = LABELS)
sn.heatmap(df_cm,annot=True,fmt="")
plt.show()

## Method 3: CNN

In [ ]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.constraints import maxnorm
from keras.optimizers import SGD

In [ ]:
X_train = []
for v in dataset_train.img.values:
    X_train.append(np.array(v))
X_train = np.array(X_train)

X_test = []
for v in dataset_test.img.values:
    X_test.append(np.array(v))
X_test = np.array(X_test)

y_train = to_categorical(dataset_train.label.values)
y_test = to_categorical(dataset_test.label.values)

#### Compile Model

In [ ]:
EPOCHS = 25
L_RATE = 0.01
DECAY = L_RATE/EPOCHS

sgd = SGD(lr=L_RATE, momentum=0.9, decay=DECAY, nesterov=False)

# Create Keras model
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='valid', activation='relu',input_shape=(32,32,3)))
model.add(Dropout(0.2))

model.add(Convolution2D(32, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(Dropout(0.2))

model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(ZeroPadding2D((1, 1)))

model.add(Convolution2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(Dropout(0.2))

model.add(Convolution2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(ZeroPadding2D((1, 1)))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# model.summary()

#### Fit

In [ ]:
model.fit(X_train, y_train, batch_size=32, nb_epoch=25,verbose=1)

#### Evaluate

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
score

#### Predict

In [ ]:
preds = []
for pred in model.predict(X_test):
    preds.append(pred.argmax())
    
y_test_labels = []
for y in y_test:
    y_test_labels.append(y.argmax())

In [ ]:
cm = confusion_matrix(preds,y_test_labels)
df_cm = pd.DataFrame(cm, index = LABELS,
                  columns = LABELS)
sn.heatmap(df_cm,annot=True,fmt="")
plt.show()